## 관람객 평점&리뷰 추출

1. 리뷰 페이지 링크 전달
2. 그 페이지의 전체 리뷰&별점 추출 (함수)
3. 다음 페이지로 넘어가서 똑같이 추출 (함수)

In [2]:
import requests
from bs4 import BeautifulSoup
import re

def get_review_viewer(box):
    point = box.select_one('div em').text
    try:
        review = box.select_one('div p span a')['data-src']
    except:
        review = box.select_one('div p').text
    return { "point": point, "review": review.replace('\n관람객','').replace('\t', '').replace('\n', '').replace('\r', '')}

def get_page_reviews(url) :
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    elements = soup.select('div.score_result ul li')
    return  [get_review_viewer(element)   for element in elements ]      

In [3]:
#전체 리뷰 추출
review_list=[]
for i in range(1, 2732) :
    review_list.extend(get_page_reviews('https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=192608&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='+str(i)))

In [14]:
#데이터 프레임 저장
import pandas as pd
review_df = pd.DataFrame(review_list)
review_df.to_csv('범죄도시2_네이버영화_관람객리뷰.csv')
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27309 entries, 0 to 27308
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   point   27309 non-null  object
 1   review  27309 non-null  object
dtypes: object(2)
memory usage: 426.8+ KB


## 전문가 평점&리뷰 추출

In [11]:
import requests
from bs4 import BeautifulSoup
import re
def get_review_pro(box, route):
    point = box.select_one('div em').text
    if route == 'div.score_result ul li':
        review = box.select_one('div p').text
    else:
        review = box.select_one('p.tx_report').text
    return { "point": point, "review": review}

def get_page_reviews_pro(url, route) :
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    elements = soup.select(route)
    return  [get_review_pro(element, route)   for element in elements ] 

In [12]:
#전문가 리뷰 추출
review_pro_list=[]
review_pro_list.extend(get_page_reviews_pro('https://movie.naver.com/movie/bi/mi/point.naver?code=192608#', 'div.reporter ul li'))
review_pro_list.extend(get_page_reviews_pro('https://movie.naver.com/movie/bi/mi/point.naver?code=192608#', 'div.score_result ul li'))
print(len(review_pro_list))

9


In [13]:
#데이터 프레임 저장
import pandas as pd
review_pro_df = pd.DataFrame(review_pro_list)
review_pro_df.to_csv('범죄도시2_네이버영화_전문가리뷰.csv')
review_pro_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   point   9 non-null      object
 1   review  9 non-null      object
dtypes: object(2)
memory usage: 272.0+ bytes


## 자연어처리 > 토큰화